In [ ]:
!pip install transformers datasets torch scikit-learn wandb

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
from sklearn.model_selection import train_test_split

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-bert")
model = AutoModelForMaskedLM.from_pretrained("l3cube-pune/hing-bert")

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
df = load_dataset("Tirath5504/IPD-Text-Hinglish")

Generating train split:   0%|          | 0/11998 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
df['train'][:10]

{'text': ['Bhai 60sal pehle desh me kya tha pehle pta kro desh waise ka waise kaise hai',
  'chutiya friday ko isliye releae krte kyoki week end ke 3 din milte h to log free hokr film dekh sake..!! hd h yrr chutiyapa ki',
  'जय मोदीराज',
  'UPA walo ne bahot kuch kr diya tha desh k liye',
  'Pan ko Aadhar se link ki zarurat kuy hai? Supreme Court ne ye bola hi nahi',
  'काकर पाथर जोड़ के मस्जिद लई बनाय।\n\nता चढ़ि मुल्ला बांघ दे , क्या बहरा हुआ खुदाय। \n\nयह मैं नहीं कबीर ने कहा था।',
  'Us raat tere papa k jageh mera sperm gya tha use of technology \nAur tere papa ko pta v na chala 😂😂😂\nChutiya',
  'गटर के कीड़े',
  'Waise bandhu jet lag se bachne ke liye Raat ko 10 baje ke baad so jao',
  'get your love back +91-(9815215009) and #all_problems_solution_with_in_72_hours one call change life \nसमस्या चाहे कैसी भी हो जड़ से खत्म।\nबड़ी से बड़ी समस्या का समाधान।101%\nअपने मन चाहे प्यार को पाना, काम कारोबार, किया कराया, सोतन से छुटकारा, पति - पत्नी में अनबन, +91-(9815215009)'],
 'hate_score': [

In [ ]:
df.shape

{'train': (11998, 2), 'test': (3000, 2)}

In [ ]:
import re

def preprocess_function(examples):
    latin_only_pattern = re.compile(r'[^\x00-\x7F]+')

    def filter_non_latin(text):
        return latin_only_pattern.sub('', text)

    filtered_texts = [filter_non_latin(text) for text in examples['text']]

    tokenized_inputs = tokenizer(filtered_texts, truncation=True, padding=True)

    if 'hate_score' in examples:
        label2id = {"NAG": 0, "CAG": 1, "OAG": 2}
        mapped_labels = [label2id[label] for label in examples['hate_score']]
        return {**tokenized_inputs, 'labels': mapped_labels}

    return tokenized_inputs

In [ ]:
tokenized_dataset = df.map(preprocess_function, batched=True)

Map:   0%|          | 0/11998 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
train_dataset = tokenized_dataset["train"].shuffle(seed=107)
# validation_dataset = tokenized_dataset["validation"].shuffle(seed=107)

In [ ]:
train_dataset_split = train_dataset.train_test_split(test_size=0.2, seed=42)

In [ ]:
train_dataset = train_dataset_split['train']
validation_dataset = train_dataset_split['test']

In [ ]:
import os
import wandb
os.environ["WANDB_API_KEY"]="59d808e9fcd215f43e751c8077b674e585909971"
os.environ["WANDB_ENTITY"]="tirath-bhathawala"
os.environ["WANDB_PROJECT"]="IPD-HINGLISH"

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    output = {
        "overall-f1": f1_score(labels, predictions, average='micro'),
        "overall-accuracy": accuracy_score(labels, predictions),
        "overall-precision": precision_score(labels, predictions, average='micro'),
        "overall-recall": recall_score(labels, predictions, average='micro'),
    }

    return output

In [ ]:
id2label = {0: "NAG", 1: "CAG", 2: "OAG"}
label2id = {"NAG": 0, "CAG": 1, "OAG": 2}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(
    "distilroberta-base", num_labels=3, id2label=id2label, label2id=label2id
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="hinglish-multiclassifier-run-2",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    report_to="wandb",
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()
wandb.finish()

Epoch,Training Loss,Validation Loss,Overall-f1,Overall-accuracy,Overall-precision,Overall-recall
1,0.740800,0.987790,0.572083,0.572083,0.572083,0.572083
2,0.666300,1.065389,0.565000,0.565000,0.565000,0.565000
3,0.709200,0.951782,0.585833,0.585833,0.585833,0.585833
4,0.682400,1.008032,0.565000,0.565000,0.565000,0.565000
5,0.583100,1.108733,0.563750,0.563750,0.563750,0.563750


In [ ]:
test_dataset = tokenized_dataset["test"].shuffle(seed=23)

In [ ]:
tester = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (

In [ ]:
wandb.init(project="IPD", entity="tirath-bhathawala", name="test-run")

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-1.0123566 ,  1.0892237 ,  0.47419333],
       [-0.7811175 ,  0.8043531 ,  0.43311897],
       [-1.494933  ,  0.88358396,  1.0755714 ],
       ...,
       [-1.0158541 ,  0.28966266,  1.3991026 ],
       [-0.4739451 ,  0.5695502 ,  0.3262117 ],
       [-0.09597836,  0.66075975, -0.1823809 ]], dtype=float32), label_ids=array([1, 1, 2, ..., 2, 0, 1]), metrics={'test_loss': 0.8985750079154968, 'test_overall-f1': 0.5633333333333334, 'test_overall-accuracy': 0.5633333333333334, 'test_overall-precision': 0.5633333333333334, 'test_overall-recall': 0.5633333333333334, 'test_runtime': 46.4495, 'test_samples_per_second': 64.586, 'test_steps_per_second': 4.047})

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (

In [ ]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="/content/hinglish-multiclassifier-run-2/checkpoint-1200",
    device=0 if torch.cuda.is_available() else -1
)

In [ ]:
test_string = "tere baap ko bula gaand marta hu uski"
result = classifier(test_string)
print(result)

[{'label': 'OAG', 'score': 0.7999101281166077}]


In [ ]:
test_string = "namaste bhailog"
result = classifier(test_string)
print(result)

[{'label': 'NAG', 'score': 0.9481797814369202}]


In [ ]:
!zip -r '/content/speech-multiclassifier-run-2/checkpoint-9576'.zip '/content/speech-multiclassifier-run-2/checkpoint-9576'

  adding: content/speech-multiclassifier-run-2/checkpoint-9576/ (stored 0%)
  adding: content/speech-multiclassifier-run-2/checkpoint-9576/merges.txt (deflated 53%)
  adding: content/speech-multiclassifier-run-2/checkpoint-9576/config.json (deflated 50%)
  adding: content/speech-multiclassifier-run-2/checkpoint-9576/tokenizer_config.json (deflated 76%)
  adding: content/speech-multiclassifier-run-2/checkpoint-9576/tokenizer.json (deflated 72%)
  adding: content/speech-multiclassifier-run-2/checkpoint-9576/model.safetensors (deflated 7%)
  adding: content/speech-multiclassifier-run-2/checkpoint-9576/optimizer.pt (deflated 30%)
  adding: content/speech-multiclassifier-run-2/checkpoint-9576/vocab.json (deflated 59%)
  adding: content/speech-multiclassifier-run-2/checkpoint-9576/trainer_state.json (deflated 73%)
  adding: content/speech-multiclassifier-run-2/checkpoint-9576/scheduler.pt (deflated 56%)
  adding: content/speech-multiclassifier-run-2/checkpoint-9576/training_args.bin (deflate